In [5]:
# IEEE 754 Material Design GUI for Jupyter Notebook (Working Version)
# ====================================================================

import struct
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# ==================== CORE CONVERTER FUNCTIONS ====================
# [Converter functions remain the same as before]
def decimal_to_ieee754(decimal_num):
    """Convert decimal to IEEE 754 32-bit binary string."""
    if np.isnan(decimal_num):
        return "0 11111111 11111111111111111111111"
    if np.isinf(decimal_num):
        if decimal_num > 0:
            return "0 11111111 00000000000000000000000"
        else:
            return "1 11111111 00000000000000000000000"
    if decimal_num == 0.0:
        return "1 00000000 00000000000000000000000" if str(decimal_num).startswith('-') else "0 00000000 00000000000000000000000"
    
    sign = 0 if decimal_num >= 0 else 1
    packed = struct.pack('!f', float(abs(decimal_num)))
    bits = struct.unpack('!I', packed)[0]
    
    sign_bit = (bits >> 31) & 0x1
    exponent = (bits >> 23) & 0xFF
    mantissa = bits & 0x7FFFFF
    
    return f"{sign_bit} {format(exponent, '08b')} {format(mantissa, '023b')}"

def ieee754_to_decimal(ieee754_binary):
    """Convert IEEE 754 binary string to decimal."""
    ieee754_binary = ieee754_binary.replace(" ", "").strip()
    if len(ieee754_binary) != 32:
        raise ValueError("Must be exactly 32 bits")
    
    sign_bit = int(ieee754_binary[0])
    exponent = int(ieee754_binary[1:9], 2)
    mantissa = int(ieee754_binary[9:], 2)
    
    if exponent == 255:
        if mantissa == 0:
            return float('inf') if sign_bit == 0 else float('-inf')
        else:
            return float('nan')
    
    sign = -1 if sign_bit == 1 else 1
    
    if exponent == 0:
        exponent_value = -126
        mantissa_value = 0.0
        for i, bit in enumerate(ieee754_binary[9:]):
            if bit == '1':
                mantissa_value += 2 ** (-(i + 1))
    else:
        exponent_value = exponent - 127
        mantissa_value = 1.0
        for i, bit in enumerate(ieee754_binary[9:]):
            if bit == '1':
                mantissa_value += 2 ** (-(i + 1))
    
    return sign * (2 ** exponent_value) * mantissa_value

def decode_components(binary_str):
    """Decode IEEE 754 binary into readable components."""
    binary_str = binary_str.replace(" ", "").strip()
    if len(binary_str) != 32:
        return {"error": "Invalid length"}
    
    sign = int(binary_str[0])
    exponent = int(binary_str[1:9], 2)
    mantissa = binary_str[9:]
    
    return {
        "sign": f"{'0 (+)' if sign == 0 else '1 (-)'}",
        "exponent": f"{binary_str[1:9]} (biased: {exponent}) → actual: {exponent - 127}",
        "mantissa": f"{mantissa[:4]}...{mantissa[-4:]} (23 bits)",
        "full": f"{binary_str}"
    }

# ==================== MATERIAL DESIGN GUI ====================

material_css = """
<style>
    .material-card { background: #ffffff; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1), 0 1px 2px rgba(0,0,0,0.08); padding: 24px; margin: 16px 0; font-family: 'Roboto', 'Noto Sans', sans-serif; }
    .material-title { color: #1976d2; font-size: 20px; font-weight: 500; margin-bottom: 16px; letter-spacing: 0.5px; }
    .material-subtitle { color: #424242; font-size: 14px; font-weight: 500; margin: 16px 0 8px 0; }
    .material-result { background-color: #f5f5f5; border-left: 4px solid #1976d2; padding: 16px; margin: 16px 0; border-radius: 4px; font-family: 'Courier New', monospace; font-size: 14px; word-break: break-all; }
    .material-error { background-color: #ffebee; border-left: 4px solid #d32f2f; color: #d32f2f; padding: 16px; margin: 16px 0; border-radius: 4px; }
    .material-info { background-color: #e3f2fd; border-left: 4px solid #1976d2; padding: 16px; margin: 16px 0 8px 0; border-radius: 4px; font-size: 13px; }
    .component-grid { display: grid; grid-template-columns: 1fr 3fr; gap: 8px; margin-top: 12px; }
    .component-label { font-weight: 500; color: #424242; padding: 8px 0; }
    .component-value { font-family: 'Courier New', monospace; background: #f5f5f5; padding: 8px; border-radius: 4px; word-break: break-all; }
</style>
"""

display(HTML(material_css))

# Main container
main_container = widgets.VBox(layout=widgets.Layout(width='100%', max_width='600px', margin='0 auto', padding='16px'))

# Title
title_label = widgets.HTML(value='<div class="material-title">IEEE 754 Floating-Point Converter</div>')

# Direction selector
direction_tabs = widgets.ToggleButtons(
    options=['Decimal → IEEE 754', 'IEEE 754 → Decimal'],
    button_style='',
    style={'button_width': 'auto', 'font_weight': '500'},
    layout=widgets.Layout(width='100%', margin='8px 0')
)

# Input field
input_field = widgets.Text(
    placeholder='Enter value...',
    continuous_update=False,
    layout=widgets.Layout(width='100%', height='48px', padding='8px')
)

# Convert button
convert_btn = widgets.Button(
    description='CONVERT',
    button_style='primary',
    layout=widgets.Layout(width='100%', height='48px', margin='8px 0')
)

# FIXED: Results container with explicit layout and initial content
results_container = widgets.VBox(
    layout=widgets.Layout(
        width='100%',
        min_height='120px',
        margin='16px 0',
        border='1px dashed #e0e0e0',
        border_radius='4px',
        padding='16px'
    )
)

# Set initial content so the container is visible
results_container.children = [
    widgets.HTML(value='<div style="color: #757575; font-style: italic;">Results will appear here...</div>')
]

# Info box
info_box = widgets.HTML(value="""
<div class="material-info">
<strong>Format:</strong> IEEE 754 Single Precision (32-bit)<br>
<strong>Structure:</strong> 1 sign bit | 8 exponent bits | 23 mantissa bits<br>
<strong>Examples:</strong> 5.625, -3.14, inf, nan
</div>
""")

# Assemble GUI
main_container.children = [
    widgets.HTML('<div class="material-card">'),
    title_label,
    info_box,
    direction_tabs,
    input_field,
    convert_btn,
    results_container,
    widgets.HTML('</div>')
]

# ==================== FIXED EVENT HANDLERS ====================

def update_results(content_widgets):
    """Reliably update the results container."""
    results_container.children = content_widgets

def on_convert_clicked(b):
    """Handle convert button click."""
    direction = direction_tabs.value
    user_input = input_field.value.strip()
    
    try:
        if direction == 'Decimal → IEEE 754':
            # Parse input
            if user_input.lower() in ['inf', 'infinity', '+inf']:
                num = float('inf')
            elif user_input.lower() in ['-inf', '-infinity']:
                num = float('-inf')
            elif user_input.lower() in ['nan']:
                num = float('nan')
            else:
                num = float(user_input)
            
            # Convert
            binary = decimal_to_ieee754(num)
            decimal_back = ieee754_to_decimal(binary)
            components = decode_components(binary)
            
            # Build output widgets (FIXED: Direct assignment instead of display())
            output_widgets = [
                widgets.HTML(value=f'<div class="material-subtitle">Result</div>'),
                widgets.HTML(value=f'<div class="material-result">{binary}</div>'),
                widgets.HTML(value=f'<div class="material-subtitle">Component Breakdown</div>'),
                widgets.HTML(value=f"""
                <div class="component-grid">
                    <div class="component-label">Sign:</div><div class="component-value">{components["sign"]}</div>
                    <div class="component-label">Exponent:</div><div class="component-value">{components["exponent"]}</div>
                    <div class="component-label">Mantissa:</div><div class="component-value">{components["mantissa"]}</div>
                </div>
                """),
                widgets.HTML(value=f'<div class="material-subtitle">Round-trip Verification</div>'),
                widgets.HTML(value=f'<div class="material-info">Original: {num}<br>Converted back: {decimal_back}</div>')
            ]
            
        else:  # IEEE 754 → Decimal
            binary = user_input.replace(" ", "").strip()
            if len(binary) != 32:
                raise ValueError("Must be exactly 32 bits")
            
            decimal_val = ieee754_to_decimal(binary)
            components = decode_components(binary)
            
            output_widgets = [
                widgets.HTML(value=f'<div class="material-subtitle">Decimal Value</div>'),
                widgets.HTML(value=f'<div class="material-result">{decimal_val}</div>'),
                widgets.HTML(value=f'<div class="material-subtitle">Component Breakdown</div>'),
                widgets.HTML(value=f"""
                <div class="component-grid">
                    <div class="component-label">Sign:</div><div class="component-value">{components["sign"]}</div>
                    <div class="component-label">Exponent:</div><div class="component-value">{components["exponent"]}</div>
                    <div class="component-label">Mantissa:</div><div class="component-value">{components["mantissa"]}</div>
                </div>
                """)
            ]
            
    except Exception as e:
        output_widgets = [
            widgets.HTML(value=f'<div class="material-error">❌ Error: {str(e)}</div>')
        ]
    
    # FIXED: Update container children (more reliable than with/display)
    update_results(output_widgets)

# Wire up events
convert_btn.on_click(on_convert_clicked)

# FIXED: Proper observe callback that ignores the change dict
def handle_input_change(change):
    """Handle Enter key press in input field."""
    if change['name'] == 'value':
        on_convert_clicked(None)

input_field.observe(handle_input_change, names='value')

# ==================== DISPLAY GUI ====================
display(main_container)